In [587]:
import pandas as pd
import networkx as nw
import sys
import time

In [588]:
df = pd.read_csv(r"C:\Users\Tank.bnd\Desktop\AI_project\search-blaster\Flight_Data1.csv")
df

,Airline,SourceAirport,DestinationAirport,SourceAirport_City,SourceAirport_Country,SourceAirport_Latitude,SourceAirport_Longitude,SourceAirport_Altitude,DestinationAirport_City,DestinationAirport_Country,DestinationAirport_Latitude,DestinationAirport_Longitude,DestinationAirport_Altitude,Distance,FlyTime,Price
0,Pegasus Airlines,Sabiha Gökçen International Airport,Imam Khomeini International Airport,Istanbul,Turkey,40.898602,29.309200,312,Tehran,Iran,35.416100,51.152199,3305,1998.541333,2.624833,271.489760
1,Turkish Airlines,Atatürk International Airport,Imam Khomeini International Airport,Istanbul,Turkey,40.976898,28.814600,163,Tehran,Iran,35.416100,51.152199,3305,2040.978811,2.882362,300.589499
2,Emirates,Dubai International Airport,Imam Khomeini International Airport,Dubai,United Arab Emirates,25.252800,55.364399,62,Tehran,Iran,35.416100,51.152199,3305,1199.863567,1.536046,210.215879
3,Etihad Airways,Abu Dhabi International Airport,Imam Khomeini International Airport,Abu Dhabi,United Arab Emirates,24.433001,54.651100,88,Tehran,Iran,35.416100,51.152199,3305,1266.681453,2.144199,175.864733
4,Air Arabia,Sharjah International Airport,Imam Khomeini International Airport,Sharjah,United Arab Emirates,25.328600,55.517200,111,Tehran,Iran,35.416100,51.152199,3305,1196.902147,1.772627,174.332843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6831,Kenmore Air,Boeing Field King County International Airport,William R Fairchild International Airport,Seattle,United States,47.529999,-122.302002,21,Port Angeles,United States,48.120201,-123.500000,291,110.930397,0.161072,31.661891
6832,Kenmore Air,Boeing Field King County International Airport,Orcas Island Airport,Seattle,United States,47.529999,-122.302002,21,Eastsound,United States,48.708199,-122.910004,31,138.564899,0.899144,26.201671
6833,Kenmore Air,William R Fairchild International Airport,Boeing Field King County International Airport,Port Angeles,United States,48.120201,-123.500000,291,Seattle,United States,47.529999,-122.302002,21,110.930397,0.751768,99.285196
6834,Kenmore Air,Friday Harbor Airport,Boeing Field King County International Airport,Friday Harbor,United States,48.521999,-123.024002,113,Seattle,United States,47.529999,-122.302002,21,122.677708,1.119010,19.599675


In [589]:
df1 = df.drop_duplicates('SourceAirport')
df2 = df.drop_duplicates('DestinationAirport')
df1.set_index('SourceAirport',inplace=True)
df2.set_index('DestinationAirport',inplace=True)

In [590]:
G = nw.DiGraph()

In [591]:
G = nw.from_pandas_edgelist(df, source = 'SourceAirport', target = 'DestinationAirport', edge_attr = ['Distance','FlyTime','Price','SourceAirport_City','SourceAirport_Country','DestinationAirport_City','DestinationAirport_Country', 'SourceAirport_Latitude', 'SourceAirport_Longitude', 'SourceAirport_Altitude', 'DestinationAirport_Latitude','DestinationAirport_Longitude', 'DestinationAirport_Altitude'])

In [592]:
source_and_destination_airports = input()
airports = source_and_destination_airports.split(' - ')
start_node , target_node = airports[0],airports[1]

In [593]:
def dijkstra_algorithm(graph, start_node):
    unvisited_nodes = list(graph.nodes())
    shortest_path = {}
    previous_nodes = {}
    # using max_value to initialize the "infinity" value of the unvisited nodes   
    max_value = sys.maxsize
    for node in unvisited_nodes:
        shortest_path[node] = max_value
    # initializing the starting node's value with 0   
    shortest_path[start_node] = 0
    while unvisited_nodes:
        current_min_node = None
        for node in unvisited_nodes:  # Iterate over the nodes
            if current_min_node is None:
                current_min_node = node
            elif shortest_path[node] < shortest_path[current_min_node]:
                current_min_node = node
        # retrieving the current node's neighbors and updating their distances
        neighbors = list(graph.adj[current_min_node])
        for neighbor in neighbors:
            value = graph.get_edge_data(current_min_node, neighbor)
            tentative_value = shortest_path[current_min_node] + value.get('Distance')
            if tentative_value < shortest_path[neighbor]:
                shortest_path[neighbor] = tentative_value
                # updating the best path to the current node
                previous_nodes[neighbor] = current_min_node
        unvisited_nodes.remove(current_min_node)
    return previous_nodes, shortest_path   

In [594]:
def dijkstra_calculate_result(previous_nodes, start_node, target_node):
    path = []
    node = target_node

    while node != start_node:
        path.append(node)
        node = previous_nodes[node]
 
    path.append(start_node)
    path.reverse()
    return path

In [595]:
def dijkstra_write_file(graph, path, target_node, shortest_path, execution_time):
    total_price = 0
    total_time = 0
    f = open("Blaster-UIAI4021-PR1-Q1.txt", "w")
    f.write("dijkstra Algorithm\n"
            "Execution Time: {}\n"
            ".-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-\n".format(execution_time))
    for i in range(len(path)-1):
        value = graph.get_edge_data(path[i], path[i + 1])
        total_price = total_price + value.get('Price')
        total_time = total_time + value.get('FlyTime')
        f.write("Flight #{0}:\n"
                "From: {1}-{2}, {3}\n"
                "To: {4}-{5}, {6}\n"
                "Duration: {7}km\n"
                "Time: {8}h\n"
                "Price: {9}$"
                .format(i + 1, value.get('SourceAirport_City'), path[i], value.get('SourceAirport_Country'),
                        value.get('DestinationAirport_City'), path[i + 1], value.get('DestinationAirport_Country'),
                        round(value.get('Distance')),
                        round(value.get('FlyTime')),
                        round(value.get('Price'))))
        f.write("\n----------------------------\n")
    f.write("Total Price: {}$\n"
            "Total Duration: {}km\n"
            "Total Time: {}h".format(round(total_price), round(shortest_path[target_node]), round(total_time)))
    f.close()
    

In [596]:
dijkstra_start_time = time.time()
previous_nodes, shortest_path = dijkstra_algorithm(graph=G, start_node=start_node)
path = dijkstra_calculate_result(previous_nodes, start_node=start_node, target_node=target_node)
dijkstra_end_time = time.time()

In [597]:
dijkstra_write_file(graph=G, path=path, target_node=target_node, shortest_path=shortest_path, execution_time= dijkstra_end_time - dijkstra_start_time)

In [598]:
def a_star_algorithm(graph, start, target):
    #target lat,long,alt
    row_t = df2.loc[target]
    D_La = row_t['DestinationAirport_Latitude']
    D_Lo = row_t['DestinationAirport_Longitude']
    D_A = row_t['DestinationAirport_Altitude']

    #open_list = list of visited nodes with not always inspected adjs
    #It starts off with the start node 
    #closed_lst = list of visited nodes with always inspected adjs
    open_list = {start}
    closed_list = set([])

    #dis from start to all other nodes
    dis = {start: 0}

    #adj contains adj dic of all nodes
    adj = {start: start}

    while open_list:
        n = None

        #a node with the lowest value
        for v in open_list:
            row_v = df1.loc[v]
            S_La = row_v['SourceAirport_Latitude']
            S_Lo = row_v['SourceAirport_Longitude']
            S_A = row_v['SourceAirport_Altitude']
            v_h = ((S_La - D_La) ** 2 + (S_Lo - D_Lo) ** 2 + (S_A - D_A) ** 2) ** 0.5
            
    
            if n is None or dis[v] + v_h < dis[n] + n_h:
                n = v
                n_h = v_h

        if n is None:
            #Path does not exist
            return None

        #if the current node is target
        if n == target:
            reconst_path = []

            while adj[n] != n:
                reconst_path.append(n)
                n = adj[n]
            reconst_path.append(start)
            reconst_path.reverse()
            #found path
            return reconst_path

        #for all the neighbors of the current node:
        neighbors = list(graph.adj[n])
        for m in neighbors:
            weight = graph.get_edge_data(n, m).get('Distance')
            if m not in open_list and m not in closed_list:
                open_list.add(m)
                adj[m] = n
                dis[m] = dis[n] + weight

            else:
                if dis[m] > dis[n] + weight:
                    dis[m] = dis[n] + weight
                    adj[m] = n
                    if m in closed_list:
                        closed_list.remove(m)
                        open_list.add(m)
                        
        open_list.remove(n)
        closed_list.add(n)

    #Path does not exist
    return None

In [599]:
a_start_time = time.time()
res = a_star_algorithm(G, start_node, target_node)
a_end_time = time.time()

In [600]:
a_total_price = 0
a_total_time = 0
a_total_duration = 0
f = open("Blaster-UIAI4021-PR1-Q1.txt", "a")
f.write("\n\nA* Algorithm\n"
        "Execution Time: {}\n"
        ".-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-\n".format(a_end_time - a_start_time))
for i in range(len(res) - 1):
    value = G.get_edge_data(res[i], res[i + 1])
    a_total_price = a_total_price + value.get('Price')
    a_total_time = a_total_time + value.get('FlyTime')
    a_total_duration = a_total_duration + value.get('Distance')
    f.write("Flight #{0}:\n"
            "From: {1}-{2}, {3}\n"
            "To: {4}-{5}, {6}\n"
            "Duration: {7}km\n"
            "Time: {8}h\n"
            "Price: {9}$"
            .format(i + 1, value.get('SourceAirport_City'), res[i], value.get('SourceAirport_Country'),
                    value.get('DestinationAirport_City'), res[i + 1], value.get('DestinationAirport_Country'),
                    round(value.get('Distance')),
                    round(value.get('FlyTime')),
                    round(value.get('Price'))))
    f.write("\n----------------------------\n")
f.write("Total Price: {}$\n"
        "Total Duration: {}km\n"
        "Total Time: {}h".format(round(a_total_price), round(a_total_duration), round(a_total_time)))
f.close()